# Limpieza de datos en el dataset base de matrícula estudiantes en educación superior

### 1) Cargamos el dataset

In [ ]:
import pandas as pd

# Configuración para mostrar todas las columnas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# URL cruda del archivo CSV en GitHub
url = 'https://github.com/jcastilloc168/PythonUDD_ProyectoFinal/raw/main/data/BaseDefinitivaINDICES-2005-2024.csv'

# Leer el archivo CSV desde GitHub
df = pd.read_csv(url, delimiter=';')

### 2) Tratamiento de valores nulos

2.1 **Eliminación de columnas**. Segun la información proporcionada por el notebook exploratorio del dataset, al tener una porcentaje elevado de valores nulos. Por otra parte, se han eliminado columnas que no son significativas para el estudio que se esta realizando, tales como, nombre del campus, código del campus y codigo SIES. En resumen, las columnas no consideradas son las siguientes:

| Nombre del Campo                                      | Valores Nulos | Porcentaje de Valores Nulos (%) |
|-------------------------------------------------------|---------------|-----------------------------|
| Mínimo Puntaje Ranking                                | 211823        | 91.013109                   |
| Máximo Puntaje Ranking                                | 211810        | 91.007523                   |
| Promedio Puntaje Ranking                              | 211667        | 90.946081                   |
| Mínimo Puntaje NEM                                    | 210787        | 90.567975                   |
| Máximo Puntaje NEM                                    | 210774        | 90.562390                   |
| Promedio Puntaje NEM                                  | 210589        | 90.482901                   |
| Mención o Especialidad                                | 200534        | 86.162611                   |
| Puntaje de corte (promedio de la carrera)             | 196798        | 84.557380                   |
| Puntaje de corte (primer seleccionado)                | 196702        | 84.516132                   |
| Puntaje de corte (último seleccionado)                | 194667        | 83.641762                   |
| Mínimo Puntaje (promedio matemáticas y lenguaje)      | 191162        | 82.135783                   |
| Promedio Puntaje (promedio matemáticas y lenguaje)    | 191004        | 82.067896                   |
| Máximo Puntaje (promedio matemáticas y lenguaje)      | 190712        | 81.942433                   |
| Nº Alumnos Ingreso Via PSU o PDT                      | 189276        | 81.325433                   |
| Nº Alumnos Ingreso Otra Via                           | 155281        | 66.718943                   |
| Nombre del Campus                                     | 122362        | 52.574773                   |
| Cód. Campus                                           | 121914        | 52.382282                   |
| Códgo SIES                                            | 87047         | 37.401123                   |

In [5]:
# Definir el rango de columnas a eliminar
columns_to_delete_range = df.loc[:, 'Máximo Puntaje (promedio matemáticas y lenguaje)':'Nº Alumnos Ingreso Otra Via'].columns

# Columnas individuales adicionales a eliminar
additional_columns_to_delete = ['Mención o Especialidad', 'Nombre del Campus', 'Cód. Campus', 'Códgo SIES']

# Combinar ambas listas de columnas a eliminar
columns_to_delete = list(columns_to_delete_range) + additional_columns_to_delete

# Eliminar las columnas del DataFrame
df.drop(columns=columns_to_delete, inplace=True)

# Imprimir las columnas eliminadas
print("Columnas eliminadas:", columns_to_delete)


Columnas eliminadas: ['Máximo Puntaje (promedio matemáticas y lenguaje)', 'Promedio Puntaje (promedio matemáticas y lenguaje)', 'Mínimo Puntaje (promedio matemáticas y lenguaje)', 'Puntaje de corte (primer seleccionado)', 'Puntaje de corte (promedio de la carrera)', 'Puntaje de corte (último seleccionado)', 'Máximo Puntaje NEM', 'Promedio Puntaje NEM', 'Mínimo Puntaje NEM', 'Máximo Puntaje Ranking', 'Promedio Puntaje Ranking', 'Mínimo Puntaje Ranking', 'Nº Alumnos Ingreso Via PSU o PDT', 'Nº Alumnos Ingreso Otra Via', 'Mención o Especialidad', 'Nombre del Campus', 'Cód. Campus', 'Códgo SIES']


2.2) **Imputacion a valores vacios con la asignacion de ceros**. Para el caso del campo "Duracion (en semestres)" se ha rellenado el valor con la mediana de la columna.

Las siguientes columnas estan vacias y analizando el dataset, el correspondiente valor corresponde a cero:

| Nombre del Campo                                      | Valores Nulos | Porcentaje de Valores Nulos (%) |
|-------------------------------------------------------|---------------|-----------------------------|
| Matrícula primer año extranjeros                      | 64016         | 27.505489                   |
| Vacantes                                              | 61255         | 26.319182                   |
| Valor de arancel                                      | 59433         | 25.536330                   |
| Matrícula total extranjeros                           | 54405         | 23.375971                   |
| Valor de matrícula                                    | 53390         | 22.939860                   |
| Valor del Título                                      | 37857         | 16.265860                   |
| Matrícula primer año mujeres                          | 22340         | 9.598735                    |
| Matrícula primer año hombres                          | 22100         | 9.495615                    |
| Matrícula total mujeres                               | 3758          | 1.614684                    |
| Matrícula total hombres                               | 3347          | 1.438092                    |

In [ ]:
columns_to_fill = [
    'Matrícula primer año hombres', 'Matrícula primer año mujeres', 'Matrícula primer año extranjeros', 
    'Matrícula total hombres', 'Matrícula total mujeres', 'Matrícula total extranjeros', 
    'Valor de matrícula', 'Valor de arancel', 'Valor del Título', 'Vacantes'
]

# Rellenar valores nulos en las columnas específicas con cero
df[columns_to_fill] = df[columns_to_fill].fillna(0)

# Calcular la mediana y rellenar valores nulos en la columna "Duración (en semestres)" en una sola línea
df['Duración (en semestres)'].fillna(df['Duración (en semestres)'].median(), inplace=True)

### 3) Normalizar los valores de moneda a **pesos chilenos**

Se tiene que las columnas "Valor de matrícula", "Valor de arancel" y "Valor del Título" existe valores que estan expresados en UF, por tanto, se realiza una normalización para que todos los valores de moneda sea pesos chilenos. Para llevar a cabo lo anterior, definimos un diccionario de los valores UF a pesos chilenos entre los años 2005 al 2024 y cada valor representa el ultimo valor de UF del año repectivo y para el año 2024 corresponde a la UF al día 02.08.2024.

In [7]:
# diccionario UF a pesos chileno por año
uf_por_ano = {
    2005: 17975, 2006: 18336, 2007: 19623, 2008: 21453, 2009: 20943, 2010: 21456,
    2011: 22294, 2012: 22841, 2013: 23310, 2014: 24627, 2015: 25629, 2016: 26348,
    2017: 26798, 2018: 27566, 2019: 28310, 2020: 29070, 2021: 30992, 2022: 35111, 2023: 36789, 2024: 37579
}

# Función para limpiar y convertir a tipo numérico
def limpiar_y_convertir(columna):
    columna = columna.astype(str).str.replace(',', '', regex=False).str.strip()
    return pd.to_numeric(columna, errors='coerce')

# Aplicar la función a las columnas numéricas
columnas_numericas = ['Duración (en semestres)', 'Valor de matrícula', 'Valor de arancel', 'Valor del Título', 'Vacantes']
for columna in columnas_numericas:
    df[columna] = limpiar_y_convertir(df[columna])

# Limpiar la columna "Tipo Moneda" eliminando espacios en blanco adicionales
df['Tipo Moneda'] = df['Tipo Moneda'].str.strip()

# Recalcular valores según el tipo de moneda
def recalcular_valores(row):
    if row['Tipo Moneda'] == 'UF':
        ano = row['Año']
        if ano in uf_por_ano:
            uf_promedio = uf_por_ano[ano]
            row['Valor de matrícula'] = row['Valor de matrícula'] * uf_promedio
            row['Valor de arancel'] = row['Valor de arancel'] * uf_promedio
            row['Valor del Título'] = row['Valor del Título'] * uf_promedio
    return row

# Aplicar la función de recalculación
df = df.apply(recalcular_valores, axis=1)

### 4) Renombrar las columnas y quitar las tildes al archivo

In [ ]:
# Diccionario con los nuevos nombres de las columnas
nuevos_nombres = {
    'Año': 'Anio', 'Cód. Institución': 'Cod_Institucion', 'Nombre Institución': 'Nombre_Institucion',
    'Tipo Institución': 'Tipo_Institucion', 'Clasificación1': 'Clasificacion1', 'Clasificación2': 'Clasificacion2',
    'Clasificación3': 'Clasificacion3', 'Clasificación4': 'Clasificacion4', 'Clasificación5': 'Clasificacion5',
    'Clasificación6': 'Clasificacion6', 'Duración (en semestres)': 'Duracion', 'Valor de matrícula': 'Valor_Matricula',
    'Valor de arancel': 'Valor_Arancel', 'Valor del Título': 'Valor_Titulo', 'Vacantes': 'Vacantes',
    'Matrícula primer año hombres': 'Matricula_Primer_Anio_Hombres', 'Matrícula primer año mujeres': 'Matricula_Primer_Anio_Mujeres',
    'Matrícula primer año extranjeros': 'Matricula_Primer_Anio_Extranjeros', 'auxiliar': 'Auxiliar', 'Matrícula Primer Año': 'Matricula_Primer_Anio',
    'Matrícula total hombres': 'Matricula_Total_Hombres', 'Matrícula total mujeres': 'Matricula_Total_Mujeres',
    'Matrícula total extranjeros': 'Matricula_Total_Extranjeros', 'Matrícula Total': 'Matricula_Total',
    'Pregrado/Posgrado': 'Pregrado_Posgrado', 'Título y Grado Académico': 'Titulo_Grado_Academico',
    'Nombre de la Sede': 'Nombre_Sede', 'Comuna donde se imparte la carrera o programa': 'Comuna', 'Nombre Region': 'Nombre_Region',
    'Orden Geográfico de la Región (Norte aSur)': 'Orden_Geografico', 'Cód. Carrera': 'Cod_Carrera', 'Carrera Genérica': 'Carrera_Generica',
    'Nombre Programa': 'Nombre_Programa', 'Tipo Programa': 'Tipo_Programa', 'Area Conocimiento': 'Area_Conocimiento',
    'Tipo Carrera': 'Tipo_Carrera', 'IngresoDirecto': 'Ingreso_Directo', 'Año Inicio Actividades': 'Anio_Inicio_Actividades',
    'Cód. Sede': 'Cod_Sede', 'Tipo Moneda': 'Tipo_Moneda'
}

# Renombrar las columnas
df = df.rename(columns=nuevos_nombres)

# Función para eliminar tildes de una cadena
def remover_tildes(cadena):
    if isinstance(cadena, str):
        return cadena.translate(str.maketrans('áéíóúÁÉÍÓÚñÑ', 'aeiouAEIOUnN'))
    return cadena

df = df.applymap(remover_tildes)

### 5) Verificar la existencia de valores nulos en el dataset limpiado y de valores duplicados

In [ ]:
# Verificar valores nulos por cada columna y mostrar solo las columnas con valores nulos
null_values = df.isnull().sum()
print("\nValores nulos en cada columna del dataset:")
print(null_values)

# Mostrar las columnas con valores nulos
columns_with_nulls = null_values[null_values > 0]
print("\nColumnas con valores nulos en el dataset:")
print(columns_with_nulls)

# Verificar filas duplicadas
print("\nFilas duplicadas en el dataset:")
print(df.duplicated().sum())

### 6) Guardar el nuevo dataset cleaned

In [10]:
# Guardar el archivo actualizado
cleaned_file_path2 = 'C:/Users/jcast/Documents/PythonUDD_ProyectoFinal/data/BaseDefinitivaINDICES-2005-2024_cleaned.csv'

df.to_csv(cleaned_file_path2, sep=';', index=False, encoding='utf-8')

# Cargar el dataset clean especificando el delimitador
df_cleaned = pd.read_csv(cleaned_file_path2, sep=';', encoding='utf-8')